In [1]:
#import packages
from os import listdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode
from nipype.interfaces.fsl.preprocess import FLIRT
from nipype.interfaces.fsl.maths import MeanImage
from nipype.interfaces.fsl.utils import Merge

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI')

# Set study variables
studyhome = '/Users/catcamacho/Box/SNAP/BABIES'
#studyhome = '/share/iang/active/BABIES/BABIES_rest'
raw_data = '/Users/catcamacho/Box/SNAP/BABIES/seg_temp/BABIES_C'
output_dir = studyhome + '/processed/template'
workflow_dir = studyhome + '/workflows'
subjects_list = open(studyhome + '/misc/templatesubjects.txt').read().splitlines()

template_brain = studyhome + '/templates/T2w_BABIES_template_2mm.nii'

proc_cores = 2 # number of cores of processing for the workflows

tissue_list=['wm','gm']

In [2]:
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subj']),
                     name="infosource")
infosource.iterables = ('subj', subjects_list)

# Data grabber- select gm/wm
templates = {'struct': raw_data + '/T1{subj}/T1{subj}-5/skullstripped_anat.nii',
             'seg': raw_data + '/T1{subj}/T1{subj}-5/seg.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
substitutions = [('_subj_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

In [ ]:
def split_tissues(in_tissues):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from nibabel import load, save, Nifti1Image
    
    segmentation_nifti = load(in_tissues)
    segmentation_data = segmentation_nifti.get_data()

    gm_data = segmentation_data
    gm_data[gm_data==150] = 1
    gm_data[gm_data==10] = 0
    gm_data[gm_data==250] = 0
    gm_nifti = Nifti1Image(gm_data, segmentation_nifti.affine)
    save(gm_nifti, 'gm.nii')
    gm_file = abspath('gm.nii')
    
    wm_data = segmentation_nifti.get_data()
    wm_data[wm_data==150] = 0
    wm_data[wm_data==10] = 0
    wm_data[wm_data==250] = 1
    wm_nifti = Nifti1Image(wm_data, segmentation_nifti.affine)
    save(wm_nifti, 'wm.nii')
    wm_file = abspath('wm.nii')

    out_tissues=[wm_file, gm_file]
    
    return(out_tissues)

split_tissues = Node(Function(input_names=['in_tissues'], 
                              output_names=['out_tissues'], 
                              function=split_tissues), 
                     name='split_tissues')

register_anat = Node(FLIRT(reference=template_brain, 
                           out_matrix_file='xform.mat'), 
                     name='register_anat')

register_seg = MapNode(FLIRT(reference=template_brain,
                             apply_xfm=True),
                       name='register_seg', 
                       iterfield=['in_file'])



In [ ]:
templatetissue = Workflow(name='templatetissue')
templatetissue.connect([(infosource, selectfiles,[('subj','subj')]),
                        (selectfiles, register_anat,[('struct','in_file')]),
                        (selectfiles, split_tissues,[('seg','in_tissues')]),
                        (register_anat, register_seg, [('out_matrix_file','in_matrix_file')]),
                        (split_tissues, register_seg, [('out_tissues','in_file')]),
                        (register_anat, datasink, [('out_file','reg_anat')]),
                        (register_seg, datasink, [('out_file','reg_tissues')])
                       ])
templatetissue.base_dir = workflow_dir
templatetissue.write_graph(graph2use='flat')
templatetissue.run('MultiProc', plugin_args={'n_procs': proc_cores})

180116-07:47:28,767 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/graph.dot.png (graph2use=flat, simple_form=True).
180116-07:47:28,809 workflow INFO:
	 Workflow templatetissue settings: ['check', 'execution', 'logging', 'monitoring']
180116-07:47:28,911 workflow INFO:
	 Running in parallel.
180116-07:47:28,920 workflow INFO:
	 Currently running 0 tasks, and 16 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2
180116-07:47:28,925 workflow INFO:
	 Executing node templatetissue.selectfiles in dir: /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_040/selectfiles180116-07:47:28,925 workflow INFO:
	 Executing node templatetissue.selectfiles in dir: /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_045/selectfiles

180116-07:47:28,939 workflow INFO:
	 Running node "selectfiles" ("nipype.interfaces.io.SelectFiles").180116-07:47:28,939 workflow INFO:
	 Running node "selectfiles" ("nipype.interf

	 out_file_skullstripped_anat_flirt.nii
180116-07:47:35,12 interface DEBUG:
	 out_matrix_file_xform.mat
180116-07:47:35,15 interface DEBUG:
	 reference_/Users/catcamacho/Box/SNAP/BABIES/templates/T2w_BABIES_template_2mm.nii
180116-07:47:35,19 workflow INFO:
	 Running node "register_anat" ("nipype.interfaces.fsl.preprocess.FLIRT"), a CommandLine Interface with command:
flirt -in /Users/catcamacho/Box/SNAP/BABIES/seg_temp/BABIES_C/T1040/T1040-5/skullstripped_anat.nii -ref /Users/catcamacho/Box/SNAP/BABIES/templates/T2w_BABIES_template_2mm.nii -out skullstripped_anat_flirt.nii -omat xform.mat.
180116-07:47:35,25 interface DEBUG:
	 in_file_/Users/catcamacho/Box/SNAP/BABIES/seg_temp/BABIES_C/T1040/T1040-5/skullstripped_anat.nii
180116-07:47:35,29 interface DEBUG:
	 out_file_skullstripped_anat_flirt.nii
180116-07:47:35,32 interface DEBUG:
	 out_matrix_file_xform.mat
180116-07:47:35,35 interface DEBUG:
	 reference_/Users/catcamacho/Box/SNAP/BABIES/templates/T2w_BABIES_template_2mm.nii
180116-

	 in_file_/Users/catcamacho/Box/SNAP/BABIES/seg_temp/BABIES_C/T1039/T1039-5/skullstripped_anat.nii
180116-07:48:25,150 interface DEBUG:
	 out_file_skullstripped_anat_flirt.nii
180116-07:48:25,154 interface DEBUG:
	 out_matrix_file_xform.mat
180116-07:48:25,157 interface DEBUG:
	 reference_/Users/catcamacho/Box/SNAP/BABIES/templates/T2w_BABIES_template_2mm.nii
180116-07:48:25,160 workflow INFO:
	 Running node "register_anat" ("nipype.interfaces.fsl.preprocess.FLIRT"), a CommandLine Interface with command:
flirt -in /Users/catcamacho/Box/SNAP/BABIES/seg_temp/BABIES_C/T1039/T1039-5/skullstripped_anat.nii -ref /Users/catcamacho/Box/SNAP/BABIES/templates/T2w_BABIES_template_2mm.nii -out skullstripped_anat_flirt.nii -omat xform.mat.
180116-07:48:25,165 interface DEBUG:
	 in_file_/Users/catcamacho/Box/SNAP/BABIES/seg_temp/BABIES_C/T1039/T1039-5/skullstripped_anat.nii
180116-07:48:25,169 interface DEBUG:
	 out_file_skullstripped_anat_flirt.nii
180116-07:48:25,173 interface DEBUG:
	 out_matrix_

180116-07:49:13,21 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/skullstripped_anat_flirt.nii;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/skullstripped_anat_flirt.mat;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/xform.hdr;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/xform.img;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/xform.mat;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/xform.nii;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/xform.mat;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/_0x9b5c1fa75de5aac809c6495d05072b74_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_039/register_anat/command.txt;/Users/catcamacho/Box/SNAP/BAB

	 Found hashfiles: []
180116-07:49:17,267 workflow DEBUG:
	 Final hashfile: /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_035/register_anat/_0x8e545c1a87a9d3727e8a918eaf21b596.json
180116-07:49:17,270 workflow DEBUG:
	 updatehash=False, overwrite=None, always_run=False, hash_exists=False
180116-07:49:17,273 workflow DEBUG:
	 Node hash: 8e545c1a87a9d3727e8a918eaf21b596
180116-07:49:17,276 workflow DEBUG:
	 /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_035/register_anat/_0x8e545c1a87a9d3727e8a918eaf21b596_unfinished.json found and can_resume is True or Node is a MapNode - resuming execution
180116-07:49:17,280 workflow DEBUG:
	 Creating /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_035/register_anat
180116-07:49:17,285 workflow DEBUG:
	 writing pre-exec report to /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_035/register_anat/_report/report.rst
180116-07:49:17,291 workflow DEBUG:
	 copying files to wd [execute=Tr

	 writing pre-exec report to /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_033x/split_tissues/_report/report.rst
180116-07:49:21,335 workflow DEBUG:
	 copying files to wd [execute=True, linksonly=False]
180116-07:49:21,341 workflow INFO:
	 Running node "split_tissues" ("nipype.interfaces.utility.wrappers.Function").
180116-07:49:21,900 workflow DEBUG:
	 Needed files: /Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_033x/split_tissues/wm.nii;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_033x/split_tissues/wm.mat;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_033x/split_tissues/gm.nii;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_033x/split_tissues/gm.mat;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_033x/split_tissues/_0x949474097094956bd150167a9ea3ea2e_unfinished.json;/Users/catcamacho/Box/SNAP/BABIES/workflows/templatetissue/_subj_033x/split_tissues/_inputs.pklz;/Users/catcama

In [ ]:
tissueinfo = Node(IdentityInterface(fields=['tissue']),
                     name='tissueinfo')
tissueinfo.iterables = ('tissue', tissue_list)

select_tissue = Node(DataGrabber(infields=['tissue'], 
                                 base_directory=output_dir, 
                                 sort_filelist=True,
                                 template=output_dir+'/*/*/*/%s*.nii',
                                 field_template={'tissue_data':output_dir +'/*/*/*/%s*.nii'},
                                 template_args = dict(tissue_data=[['tissue']])),
                     name='select_tissue')

In [ ]:
merge_image = Node(Merge(dimension='t'), 
                   name='merge_image')

average_image = Node(MeanImage(), 
                     name='average_image')

In [ ]:
group_tissue = Workflow(name='group_tissue')
group_tissue.connect([(tissueinfo, select_tissue, [('tissue','tissue')]),
                      (select_tissue, merge_image, [('tissue_data','in_files')]),
                      (merge_image, average_image, [('merged_file','in_file')]),
                      (average_image, datasink, [('out_file','average_tissue')])
                     ])
group_tissue.base_dir = workflow_dir
group_tissue.write_graph(graph2use='flat')
group_tissue.run('MultiProc', plugin_args={'n_procs': proc_cores})